# Phase 1

### Importing the necessary libraries

If you get any errors like module not found then uncommenting the pip install code should help


In [170]:
#!pip install pyodbc
#!pip install pandas
import pyodbc
import pandas as pd
import numpy as np
import os.path
import calendar
print(pyodbc.drivers())

['ODBC Driver 17 for SQL Server', 'FreeTDS']


The code above should output ['ODBC Driver 17 for SQL Server', ....] or similar driver, if the steps on [this doc](https://docs.google.com/document/d/17D6-JHvKJo9K2iX5eFw4p5OSrM3dg8U_-PCOIAFNDsE/edit?usp=sharing) are followed.

### Connecting to the SQL Server Database

In [237]:
## Try to Rerun block of code if there are errors in connection
server = 'ist-rs-cent-sql.ad.syr.edu' # enter server name
database = 'Centro_Preprocessing_2020' # enter database name

# If the UID and PWD are not working you can check the document linked above for steps to create a new account
s='''DRIVER={ODBC Driver 17 for SQL Server};SERVER=ist-rs-cent-sql.ad.syr.edu;
DATABASE=Centro_Preprocessing_2020;UID=CENTDATA2;PWD=Hinds200'''
cnxn = pyodbc.connect(s)
cursor = cnxn.cursor()

### Replicating Michael's previous work

The following section replicates Michael Amoury's work on this project. First we get all route names from the database. Then for each route we get the monthly ridership, supply and, capacity data and compares these across 2019 & 2020.

In [172]:
#The following query return all distinct route names from the 2020 dataset.
route_names = pd.read_sql_query('''select DISTINCT(ROUTE_NAME) from 
                                Centro_Preprocessing_2020.dbo.FINAL_PREPROCESSED_DATA_2020''',cnxn)

#### Below is a helper function to execute SQL queries

In [188]:
# Function to run SQL queries
def query_run_helper(query_string):
    try:
        data = pd.read_sql_query(query_string,cnxn)
        return data
    except Exception as e:
        print(e)

In [173]:
# Dividing all route names into SU routes and non-SU routes
su_route_names = []
non_su_route_names = []
all_route_names = []
for row in route_names.itertuples(index=False):
    route_name = row[0].lower()
    all_route_names.append(row[0])
    if 'su' in route_name:
        su_route_names.append(row[0])
    else:
        non_su_route_names.append(row[0])

In [174]:
# Removing the route names which are present only in 2020

# Uncomment the code below if you are running this for the first time

# su_route_names.remove(['SU345','SU343'])
# su_route_names

# Analysis of the new routes will be done separately in later versions
new_su_routes = ['SU345','SU343']

#### Definitions: 
##### Ridesrship - The sum of passengers_on 
##### Supply - The number of distinct serial Numbers
##### Capacity - The percentage of passengers_in over the total vehicle seats in bus

In [175]:
# The functions given below returns Ridership, Supply, Capacity for a ROUTE NAME
def get_2020_data_for_route(route_name):
    try:
        query = '''select MONTH(TRIP_START_TIME) AS Month, Sum(PASSENGERS_ON) AS Ridership, 
        count(DISTINCT SERIAL_NUMBER) As Supply,
        ROUND(CAST((sum(passengers_in))AS FLOAT)/sum(Vehicle_seats),2)*100 as "Capacity%"
        from Centro_Preprocessing_2020.dbo.FINAL_PREPROCESSED_DATA_2020 where ROUTE_NAME = '{0}' 
        GROUP BY MONTH(TRIP_START_TIME) 
        ORDER BY MONTH(TRIP_START_TIME)'''.format(route_name)
        data = pd.read_sql_query(query,cnxn)
        return data
    except Exception as e:
        print(e)

def get_2019_data_for_route(route_name):
    try:
        query = '''select MONTH(TRIP_START_TIME) AS Month, Sum(PASSENGERS_ON) AS Ridership_19, 
        count(DISTINCT SERIAL_NUMBER) As Supply_19, 
        ROUND(CAST((sum(passengers_in))AS FLOAT)/sum(Vehicle_seats),2)*100 as "Capacity_19"
        from Centro_Preprocessing.dbo.FINAL_PREPROCESSED_DATA where ROUTE_NAME = '{0}' 
        GROUP BY MONTH(TRIP_START_TIME)
        ORDER BY MONTH(TRIP_START_TIME)
        '''.format(route_name)
        data = pd.read_sql_query(query,cnxn)
        return data
    except Exception as e:
        print(e)

### Anomaly Analysis

We define anomaly as an increase/decrease of greater than equal to 15%, in supply or ridership between 2019 and 2020. The code section below checks in which months for a route we encounter this anomaly and add that to a csv called-  "Anomaly.csv"

In [176]:
# After running this block you might get many warnings. ignore the warnings

# This block creates a dataset of anomalies. Anomalies are defined as months, when the Ridership% or Capacity% 
# exceed a thershold(which is 15 here). This also saves the dataset to disk as Anomaly.csv


if os.path.exists('./Anomaly.csv'):
    df=pd.read_csv('./Anomaly.csv')
    print(df)
else:
    combined_df=pd.DataFrame({})

    for route in all_route_names:
        print(route)
        data2020 = get_2020_data_for_route(route)
        data2019 = get_2019_data_for_route(route)
        mergedDt=pd.merge(data2020,data2019,on='Month',how='outer')
        mergedDt["Ridership%"] = (mergedDt["Ridership"]-mergedDt["Ridership_19"])/mergedDt["Ridership_19"]*100
        mergedDt["Supply%"] = (mergedDt["Supply"]-mergedDt["Supply_19"])/mergedDt["Supply_19"]*100
        supply_row=mergedDt[(mergedDt["Supply%"] > 15) | (mergedDt["Supply%"] < -15)]
        supply_row['ROUTE_NAME']=route
        ridership_row=mergedDt[(mergedDt["Ridership%"] > 15) | (mergedDt["Ridership%"] < -15)]
        ridership_row['ROUTE_NAME']=route
        combined_df = pd.concat([combined_df,supply_row,ridership_row], axis=0).drop_duplicates().reset_index(drop=True)
    combined_df['Month'] = combined_df['Month'].apply(lambda x: calendar.month_abbr[x])
    combined_df.to_csv("Anomaly.csv", index=False)

    Month  Ridership  Supply  Capacity%  Ridership_19  Supply_19  Capacity_19  \
0     Jan     2957.0   150.0       17.0        1756.0      107.0         14.0   
1     Feb     2871.0   159.0       15.0        1914.0      119.0         14.0   
2     Mar     2512.0   156.0       15.0        2005.0      113.0         16.0   
3     Apr      617.0    39.0       18.0        2604.0      136.0         16.0   
4     Mar      812.0   157.0        6.0         779.0      117.0          8.0   
..    ...        ...     ...        ...           ...        ...          ...   
425   Jan     1493.0    57.0       35.0        1272.0       56.0         31.0   
426   Mar     1940.0    52.0       55.0        5950.0      154.0         60.0   
427   Oct      213.0    40.0        8.0        4701.0      121.0         55.0   
428   Nov      145.0    38.0        5.0        3269.0       82.0         57.0   
429   Dec       47.0     8.0        6.0        2755.0       71.0         52.0   

     Ridership%    Supply% 

### Group Analysis

The following section of code helps in Analyzing groups of routes together, For example, SU-routes, All-routes and NON-SU routes. The function group_analysis given below calculates the Ridership data and, Supply data across 2019 and 2020, which is used in the visualization below.

In [180]:
# The function below returns a combined dataset for a list of routes_names, with Ridership change, Supply change,
# and average capacity.
 
def group_analysis(route_names):
    dummy_df = pd.DataFrame({"Month":[1,2,3,4,5,6,7,8,9,10,11,12]})
    combined_df = pd.DataFrame({})
    sum_dummy = pd.DataFrame({})
    for route in route_names:
        data2020 = get_2020_data_for_route(route)
        full_2020 = pd.merge(dummy_df, data2020,on='Month',how='outer')
        data2019 = get_2019_data_for_route(route)
        full_2019 = pd.merge(dummy_df, data2019,on='Month',how='outer')
        dt=pd.merge(full_2020,full_2019,on='Month',how='outer')
        sorted_dt = dt.sort_values(by=['Month'], ascending=True)
        dummy = pd.DataFrame({"Month":[1,2,3,4,5,6,7,8,9,10,11,12], 
                              "Cap":sorted_dt['Capacity%'].isnull(), "Cap19":sorted_dt['Capacity_19'].isnull()})
        dummy['Cap'] = dummy.Cap.apply(lambda x: 0 if x == True else 1)
        dummy['Cap19'] = dummy.Cap19.apply(lambda x: 0 if x == True else 1)
        sum_dummy=sum_dummy.add(dummy, fill_value=0)
        combined_df=combined_df.add(sorted_dt.drop("Month", axis=1), fill_value=0)

    combined_df['Month'] = [1,2,3,4,5,6,7,8,9,10,11,12]
    combined_df["Ridership%"] = (combined_df["Ridership"]-combined_df["Ridership_19"])/combined_df["Ridership_19"]*100
    combined_df["Supply%"] = (combined_df["Supply"]-combined_df["Supply_19"])/combined_df["Supply_19"]*100
    combined_df["Capacity%"]=combined_df["Capacity%"]/sum_dummy["Cap"]
    combined_df["Capacity_19"]=combined_df["Capacity_19"]/sum_dummy["Cap19"]
    combined_df['Month'] = combined_df['Month'].apply(lambda x: calendar.month_abbr[x])
    return combined_df

### Visualizing Route Groups 

In [181]:
# The following code block creates the necessary visualizations for group analysis

#####################################################################
%matplotlib inline
import ipywidgets as widgets
from matplotlib.pyplot import *

grp_plot_output = widgets.Output()
# The dropdown values are defined below
all_groups = ["All routes", "Non-SU", "All SU"]
all_types = ["Choose your Analysis", "Percentage Analysis", "Ridership Analysis", "Supply Analysis"]

# The code below we create the dropdown widget with above values
dropdown_grps = widgets.Dropdown(options=all_groups, description="Groups:")
dropdown_types = widgets.Dropdown(options=all_types)

# The common function to handle changes in dropdowns and refreshing the plot
def filtering(grp_name, type_name):
    grp_plot_output.clear_output()
    if grp_name=="All routes":
        if os.path.exists('./all_routes.csv'):
            df=pd.read_csv('./all_routes.csv')
        else:
            df = group_analysis(all_route_names)
    elif grp_name=="Non-SU":
        if os.path.exists('./non_su_routes.csv'):
            df=pd.read_csv('./non_su_routes.csv')
        else:
            df = group_analysis(non_su_route_names)
    else:
        if os.path.exists('./su_routes.csv'):
            df=pd.read_csv('./su_routes.csv')
        else:
            df = group_analysis(su_route_names)
    
    with grp_plot_output:
        if type_name == "Percentage Analysis":
            df.plot.bar(x = 'Month', y = ['Ridership%', 'Supply%'], rot = 40)
            show()
        elif type_name == "Ridership Analysis":
            df.plot.bar(x = 'Month', y = ['Ridership', 'Ridership_19'], rot = 40)
            show()
        elif type_name == "Supply Analysis":
            df.plot.bar(x = 'Month', y = ['Supply', 'Supply_19'], rot = 40)
            show()
    
def dropdown_grps_eventhandler(change):
    filtering(change.new, dropdown_types.value)

def dropdown_types_eventhandler(change):
    filtering(dropdown_grps.value, change.new)
    
dropdown_grps.observe(dropdown_grps_eventhandler, names='value')
dropdown_types.observe(dropdown_types_eventhandler, names ='value')

# displaying the widgets
display(dropdown_grps)
display(dropdown_types)
display(grp_plot_output)

Dropdown(description='Groups:', options=('All routes', 'Non-SU', 'All SU'), value='All routes')

Dropdown(options=('Choose your Analysis', 'Percentage Analysis', 'Ridership Analysis', 'Supply Analysis'), val…

Output()

### Visualizing Individual routes

In [183]:
# Similar to the block above we visualize percent changes for all routes

import ipywidgets as widgets
import matplotlib.pyplot as plt

plot_output = widgets.Output()
dropdown_routes = widgets.Dropdown(options=all_route_names, description="Routes:")

def filtering(route_name):
    plot_output.clear_output()
    data2020 = get_2020_data_for_route(route_name)
    data2019 = get_2019_data_for_route(route_name)
    mergedDt=pd.merge(data2020,data2019,on='Month',how='outer')
    mergedDt["Ridership%"] = (mergedDt["Ridership"]-mergedDt["Ridership_19"])/mergedDt["Ridership_19"]*100
    mergedDt["Supply%"] = (mergedDt["Supply"]-mergedDt["Supply_19"])/mergedDt["Supply_19"]*100
    mergedDt = mergedDt.sort_values(by=['Month'], ascending=True)
    mergedDt['Month'] = mergedDt['Month'].apply(lambda x: calendar.month_abbr[x])
    with plot_output:
        mergedDt.plot.bar(x = 'Month', y = ['Ridership%', 'Supply%'], rot = 40)
        show()
    
def dropdown_routes_eventhandler(change):
    filtering(change.new)
    
dropdown_routes.observe(dropdown_routes_eventhandler, names='value')
display(dropdown_routes)
display(plot_output)

Dropdown(description='Routes:', options=('Aub36', 'Aub38', 'AUB50', 'Osw46', 'SU145', 'SU243', 'SU244', 'SU245…

Output()

### Stop ID Analysis

The section below replicates Michael Amoury's work on STOP ID Analysis. The team wanted to look at the traffic on some specific bus stops.

In [356]:
# The function below returns a dataset with passengers on and off with the capacity for a specific combination of 
# route name and stop id.

def get_stop_id_data_2020(route_name, stop_id):
    try:
        query = '''select MONTH(TRIP_START_TIME) AS Month, sum(passengers_on)+sum(passengers_off) AS Capacity_20
        from Centro_Preprocessing_2020.dbo.FINAL_PREPROCESSED_DATA_2020
        where ROUTE_NAME = '{0}' and stop_id = '{1}' group by MONTH(TRIP_START_TIME);'''.format(route_name, stop_id)
        data = pd.read_sql_query(query,cnxn)
        return data
    except Exception as e:
        print(e)
def get_stop_id_data_2019(route_name, stop_id):
    try:
        query = '''select MONTH(TRIP_START_TIME) AS Month, sum(passengers_on)+sum(passengers_off) AS Capacity_19
        from Centro_Preprocessing.dbo.FINAL_PREPROCESSED_DATA
        where ROUTE_NAME = '{0}' and stop_id = '{1}' group by MONTH(TRIP_START_TIME);'''.format(route_name, stop_id)
        data = pd.read_sql_query(query,cnxn)
        return data
    except Exception as e:
        print(e)    

In [184]:
# Creating a list of stop-id and route_name combinations
combinations = [("SY54","116"), ("SY10", "116"),("SY10","121"), ("SY68", "7888"),("SY20","7888"), 
                ("SY52", "1197"),("SY72","17672"), ("SY46", "8740"), ("OSW46","8740"), ("SY48", "11707"),
                ("SU43","1683"), ("SU243", "1683"),("SU244","1683"), ("SU245","1683"), 
                ("SU344", "1683"),("SU444","1683"),("SU443", "1683"),("SY36","3761"),
                ("SY36", "6767"),("SY36","6766"), ("SY36", "1114"),("AU36", "3761"),("AU38","7934"),
               ("SY80", "3675"),("SY86","10187"), ("SY16", "47"),("SY26", "7736"),("SY26","3710"),
               ("SY26", "17815"),("SY62","17203"), ("SY74", "11542"),("SY74", "8550"),("SU44","1683"),
                ("SU45","1683")]

combined_dt = pd.DataFrame({}) # to store the resulting dataset

if os.path.exists('./stop_id_analysis.csv'):
    combined_dt=pd.read_csv('./stop_id_analysis.csv')
    print(combined_dt)
else:
    for i in combinations:
        for route in all_route_names:
            if i[0]==route.upper():
                print("{0} : {1}".format(route, i[1]))
                data2020 = get_stop_id_data_2020(route, i[1])
                data2019 = get_stop_id_data_2019(route, i[1])
                data = pd.merge(data2020, data2019, on='Month', how='outer')
                data["Route"] = i[0]
                data["STOP_ID"] = i[1]
                combined_dt = pd.concat([combined_dt, data], axis=0).reset_index(drop=True)
    combined_dt['Month'] = combined_dt['Month'].apply(lambda x: calendar.month_abbr[x])
    # we are saving the dataset onto disk without the index column
    combined_dt.to_csv("stop_id_analysis.csv", index=False)

    Month  Capacity_20  Capacity_19 Route  STOP_ID
0     Jan        206.0        224.0  SY54      116
1     Feb        212.0        275.0  SY54      116
2     Mar        183.0        181.0  SY54      116
3     Apr         98.0        271.0  SY54      116
4     May        111.0        240.0  SY54      116
..    ...          ...          ...   ...      ...
364   Oct        772.0        849.0  SU45     1683
365   Nov        125.0        466.0  SU45     1683
366   Apr          NaN        474.0  SU45     1683
367   May          NaN        192.0  SU45     1683
368   Dec          NaN         99.0  SU45     1683

[369 rows x 5 columns]


### Analysis of Stop 1683 - College Place

The next section will output a month wise capacity change for the Bus stop at College place - This stop is serviced by many routes so all routes will have separate percentages

In [187]:
stop_1683 = combined_dt[combined_dt["STOP_ID"]==1683]
final_1683 = pd.DataFrame({"Month":["Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"]})
l=[]
for route in stop_1683.Route.unique():
    temp = stop_1683[stop_1683["Route"]==route]
    temp["{}_change%".format(route)] = ((temp["Capacity_20"]) - (temp["Capacity_19"]))/(temp["Capacity_19"])
    temp= temp[["Month", "{}_change%".format(route)]]
    final_1683 = pd.merge(final_1683, temp, on='Month',how='outer')
    l.append("{}_change%".format(route))

print(final_1683)

   Month  SU43_change%  SU243_change%  SU244_change%  SU245_change%  \
0    Jan     -0.016494            NaN       0.070320            NaN   
1    Feb     -0.048499            NaN       0.093247            NaN   
2    Mar     -0.385140            NaN      -0.404709            NaN   
3    Apr           NaN            NaN            NaN            NaN   
4    May     -0.994413            NaN            NaN      -0.989071   
5    Jun           NaN            NaN            NaN            NaN   
6    Jul           NaN            NaN            NaN            NaN   
7    Aug     -0.771523      -0.413302      -0.751220      -0.966798   
8    Sep     -0.813593      -0.645747      -0.829343      -0.991740   
9    Oct     -0.857111      -0.661716      -0.856333            NaN   
10   Nov     -0.849908      -0.701525      -0.828468            NaN   
11   Dec           NaN            NaN            NaN            NaN   

    SU344_change%  SU444_change%  SU443_change%  SU44_change%  SU45_change% 

<ipython-input-187-31870d1a74ea>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp["{}_change%".format(route)] = ((temp["Capacity_20"]) - (temp["Capacity_19"]))/(temp["Capacity_19"])


Futher Stop ID analysis will be done later in the notebook!

# Phase 2

### Month/Day wise Analysis

The team wanted to extend the analysis to calculate the number of passengers per trip(a trip is defined by a distinct serial number) per route name per month/day. From this passenger data, we calculate the average and standard deviation of the number of passengers per route and per month/day.

In [189]:
# Queries to get the number of passengers per trip(a trip is defined by a distinct serial number) 
# grouped by serial_number, route_name and, month.
month_2019='''
select
route_name,
serial_number,
DATEPART(month, TRIP_START_TIME) as month,
DATEPART(year, TRIP_START_TIME) as year,
SUM((PASSENGERS_ON)) AS PASSENGERS
from
Centro_Preprocessing.dbo.FINAL_PREPROCESSED_DATA
Group by  serial_number, route_name, DATEPART(month, TRIP_START_TIME), DATEPART(year, TRIP_START_TIME)
order by route_name, SERIAL_NUMBER
'''
month_2020='''
select
route_name,
serial_number,
DATEPART(month, TRIP_START_TIME) as month,
DATEPART(year, TRIP_START_TIME) as year,
SUM((PASSENGERS_ON)) AS PASSENGERS
from
Centro_Preprocessing_2020.dbo.FINAL_PREPROCESSED_DATA_2020
Group by  serial_number, route_name, DATEPART(month, TRIP_START_TIME), DATEPART(year, TRIP_START_TIME)
order by route_name, SERIAL_NUMBER
'''

In [198]:
# Aggregate month data to get average and std. dev of the number of passengers grouped by route_name and month
def month_helper(q):
    dt=query_run_helper(q)
    mean_=dt.groupby(['route_name', 'month'], as_index=False).mean().reset_index()[['route_name', 'month', 'year', 'PASSENGERS']]
    std_=dt.groupby(['route_name', 'month'], as_index=False).var().reset_index()[['route_name', 'month', 'year', 'PASSENGERS']]
    mean_["std_dev_passengers"]=np.sqrt(std_["PASSENGERS"])
    mean_.rename(columns={'Avg_passengers': 'Avg_passengers'})
    return mean_

In [199]:
#uncomment the following lines of code to save the file to disk

# month_helper(month_2019).to_csv('month_2019.csv', index=False)
# month_helper(month_2020).to_csv('month_2020.csv', index=False)

    route_name  month    year  PASSENGERS  std_dev_passengers
0        AUB50      1  2019.0    4.406250            3.241459
1        AUB50      2  2019.0    3.517241            2.599735
2        AUB50      3  2019.0    4.347826            3.961275
3        AUB50      4  2019.0    5.611111            3.743840
4        AUB50      5  2019.0    5.111111            3.562697
..         ...    ...     ...         ...                 ...
582      SY974      6  2019.0   32.198020           12.209029
583      SY974      9  2019.0   42.044248           16.693342
584      SY974     10  2019.0   38.851240           14.552011
585      SY974     11  2019.0   39.865854           13.447440
586      SY974     12  2019.0   38.802817           14.219323

[587 rows x 5 columns]


In [365]:
# Queries to get the sum of passengers_on per serial_number, route_name per day of the year.
day_2019='''
select
route_name,
serial_number,
DATEPART(dayofyear, TRIP_START_TIME) as dayofyear,
DATEPART(year, TRIP_START_TIME) as year,
SUM((PASSENGERS_ON)) AS PASSENGERS
from
Centro_Preprocessing.dbo.FINAL_PREPROCESSED_DATA
Group by  serial_number, route_name, DATEPART(dayofyear, TRIP_START_TIME), DATEPART(year, TRIP_START_TIME)
order by route_name, SERIAL_NUMBER
'''

day_2020='''
select
route_name,
serial_number,
DATEPART(dayofyear, TRIP_START_TIME) as dayofyear,
DATEPART(year, TRIP_START_TIME) as year,
SUM((PASSENGERS_ON)) AS PASSENGERS
from
Centro_Preprocessing_2020.dbo.FINAL_PREPROCESSED_DATA_2020
Group by  serial_number, route_name, DATEPART(dayofyear, TRIP_START_TIME), DATEPART(year, TRIP_START_TIME)
order by route_name, SERIAL_NUMBER
'''

In [366]:
# Aggregate day data to get average and std. dev of the number of passengers grouped by route_name and day of the year

def day_helper(q):
    dt=query_run_helper(q)
    mean_=dt.groupby(['route_name', 'dayofyear'], as_index=False).mean().reset_index()[['route_name', 'dayofyear', 'year', 'PASSENGERS']]
    std_=dt.groupby(['route_name', 'dayofyear'], as_index=False).std().reset_index()[['route_name', 'dayofyear', 'year', 'PASSENGERS']]
    mean_["std"]=std_["PASSENGERS"]
    mean_.rename(columns={'PASSENGERS': 'Avg_passengers'})
    return mean_

In [367]:
#uncomment the following lines of code to save the file to disk

# day_helper(day_2019).to_csv('day_2019.csv', index=False)
# day_helper(day_2020).to_csv('day_2020.csv', index=False)

###  Further Analysis of Month data

We were wary that our averages were being thrown off due to zero-passenger trips so the next section extends the analysis done above by calculating how many trips in a month(for a route) are zero passenger trips. We also calculate new averages and standard deviations using only non-zero-passenger trips.

In [201]:
# Function to run an extensive on month data
# There are many trips which have zero passengers. 
# We do the average and standard deviation aggration additionally for non-zero passenger trips.
def month_extensive_analysis(query):
    dt=query_run_helper(query)
    mean_=dt.groupby(['route_name', 'month'], as_index=False).mean().reset_index()[['route_name', 'month', 'year', 'PASSENGERS']]
    std_=dt.groupby(['route_name', 'month'], as_index=False).std().reset_index()[['route_name', 'month', 'year', 'PASSENGERS']]
    mean_["std"]=std_["PASSENGERS"]
    mean_.columns= ['route_name', 'month', 'year', 'avg_psngr', 'std_psngr']
    
    #total_trips
    total = dt.groupby([ 'route_name', 'month'], as_index=False).count().reset_index()[['route_name', 'month','serial_number']]
    mean_["total_trips"]=total["serial_number"]
    
    #zero_psngr_trips
    zero = dt[dt['PASSENGERS']==0].groupby([ 'route_name', 'month'], 
                                           as_index=False).count()[['route_name', 'month','serial_number']]
    zero.columns= ['route_name', 'month', 'zero_psngr_trips']
    new_df = pd.merge(mean_, zero, how='left', on=['route_name', 'month'])
    new_df["zero_psngr_trips"] = new_df["zero_psngr_trips"].replace(np.nan, 0)
    mean_ = new_df
    mean_["%_zero_trips"]=(mean_["zero_psngr_trips"]/mean_["total_trips"])*100

    non_zero_mean=dt[dt['PASSENGERS']!=0].groupby(['route_name', 'month'], 
                                                  as_index=False).mean().reset_index()[['route_name', 'month', 'PASSENGERS']]
    non_zero_mean.columns= ['route_name', 'month', 'avg_psngr_non_zero']
    new_df = pd.merge(mean_, non_zero_mean, how='left', on=['route_name', 'month'])
    new_df["avg_psngr_non_zero"] = new_df["avg_psngr_non_zero"].replace(np.nan, 0)
    mean_ = new_df
    
    non_zero_std=dt[dt['PASSENGERS']!=0].groupby(['route_name', 'month'], 
                                                 as_index=False).std().reset_index()[['route_name', 'month', 'PASSENGERS']]
    non_zero_std.columns= ['route_name', 'month', 'std_psngr_non_zero']
    new_df = pd.merge(mean_, non_zero_std, how='left', on=['route_name', 'month'])
    new_df["std_psngr_non_zero"] = new_df["std_psngr_non_zero"].replace(np.nan, 0)
    mean_ = new_df

    return mean_

In [369]:
# month_extensive_analysis(month_2019).to_csv('month_extensive_2019.csv', index=False)
# month_extensive_analysis(month_2020).to_csv('month_extensive_2020.csv', index=False)

# Phase 3

### STOP ID Analysis

This section furthers the Stop ID Analysis. Initially the stop id involved analyzing stop_id with route_names. Due to the huge number of combinations of Stop_id and route_names, we decided to group stop ids by tract ids. Tract IDs being comparable in population will allow comparisons across tracts to be insigtful as well.

In [202]:
get_all_stop_id = '''
select stop_id, stop_lat, stop_lon, main_cross_street
from Centro_Preprocessing.dbo.FINAL_PREPROCESSED_DATA
where stop_lat is not null and stop_lon is not null and main_cross_street is not null
group by stop_id, stop_lat, stop_lon, main_cross_street;
'''
stop_id = query_run_helper(get_all_stop_id)

In [203]:
stop_id.head()

,stop_id,stop_lat,stop_lon,main_cross_street
0,7366,43.013282,-75.991600,F'ville Manlius Rd/Marangale Rd
1,8017,43.032698,-76.178577,Bellevue Ave/Duane St
2,1523,43.004144,-76.142978,S Salina St/Clarence Ave
3,4386,43.047724,-76.186349,Milton Ave/Ulster St
4,9069,43.108310,-76.173488,Flagstone Dr/Driftwood Rd


### Getting tract data from Census API

In [372]:
# We add the tract ID from Census Bureau 2020 data to each StopID. 
# check out the documentation of their api here: https://geocoding.geo.census.gov/

import urllib
import pprint
# !pip install geopy
# !pip install requests
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="geoapiExercises")

import requests

def get_tract_id(stop_id):   
    # api-endpoint
    URL = "https://geocoding.geo.census.gov/geocoder/geographies/address"

    tract_id =[]
    i=1
    for row in stop_id.itertuples(index=False):
        print(i)
        i=i+1
    #     print()
    #     print(row)

        location = geolocator.reverse(str(row[1])+","+str(row[2]))
        pprint.pprint(location.raw)
        l_df=location.raw["address"]
        PARAMS={
            "street": row[3].replace("/", " & "),
            "state":l_df.get("state", ""),
            "zip":l_df.get("postcode", ""),
            "benchmark": "Public_AR_Census2020",
            "vintage": "Census2020_Census2020",
            "format": "json"
        }
        r = requests.get(url = URL, params = PARAMS)
#         print(r.url)
        data = r.json()
#         print(data)
        df = data.get("result", {}).get("addressMatches", [])
        if len(df)>0:
            df = df[0].get("geographies", {}).get("Census Blocks", [])
            if len(df)>0:
                tract_id.append(df[0].get("TRACT", ""))
            else:
                tract_id.append("")
        else:
            tract_id.append("")
    return tract_id

In [378]:
# adding a new column "tract_id" to the stop_id.csv table
stop_id["tract_id"]=get_tract_id(stop_id)
# stop_id.to_csv('stop_ids.csv', index=False)

1
{'address': {'city': 'Syracuse',
             'country': 'United States',
             'country_code': 'us',
             'county': 'Onondaga County',
             'house_number': '4030',
             'postcode': '13206',
             'road': 'New Court Avenue',
             'state': 'New York'},
 'boundingbox': ['43.0788223', '43.0789223', '-76.1091993', '-76.1090993'],
 'display_name': '4030, New Court Avenue, Syracuse, Onondaga County, New York, '
                 '13206, United States',
 'lat': '43.0788723',
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. '
            'https://osm.org/copyright',
 'lon': '-76.1091493',
 'osm_id': 6183074272,
 'osm_type': 'node',
 'place_id': 68701319}
https://geocoding.geo.census.gov/geocoder/geographies/address?street=New+Court+Ave+%26+Thor+Metal+Product&state=New+York&zip=13206&benchmark=Public_AR_Census2020&vintage=Census2020_Census2020&format=json
{'result': {'input': {'benchmark': {'id': '2020', 'benchmarkName': 'Public_AR_Census2

KeyboardInterrupt: 

In [374]:
if os.path.exists('./stop_ids.csv'):
    df=pd.read_csv('./stop_ids.csv')
df["tract_id"] = df["tract_id"]/100
#Proportion of non-null values
1- df["tract_id"].isnull().sum()/len(df["tract_id"])

### Using ArcGIS to complete the data

We see that the census API sometimes doesn't return a tract id for some stops, so we use data from ArcGIS(provided by Prof. Baris Salman) to complement our data.

In [382]:
# !pip install openpyxl
if os.path.exists('./ArcGIS.xlsx'):
    baris=pd.read_excel('./ArcGIS.xlsx', engine = 'openpyxl')
arcgis = baris[["stop_id", "TRACTCE"]]
arcgis

,stop_id,TRACTCE
0,17,5500
1,18,14400
2,22,14600
3,28,14500
4,47,100
...,...,...
6488,18178,21601
6489,18179,21601
6490,18180,21601
6491,18181,21601


In [68]:
# for the stop ids not populated by the API, we use Prof. Baris's data to populate those
stops =df[df["tract_id"].isnull()]["stop_id"]
tracts = []
for stop in stops:
    flg=0
    for row in arcgis.itertuples(index=False):
        if(row[0]==stop):
            tracts.append(row[1]/100)
            flg=1
    if(flg==0):
        tracts.append("")

In [79]:
for (i,stop) in enumerate(stops):
    df.loc[df["stop_id"]==stop, "tract_id"]=tracts[i]
df.to_csv('stop_ids_2020.csv', index=False)

   stop_id   stop_lat   stop_lon main_cross_street  tract_id
1       18  43.086006 -76.082687  Carrier (Gate 5)     144.0
   stop_id   stop_lat  stop_lon main_cross_street  tract_id
3       28  43.077158 -76.06072     Bishop Grimes     145.0
   stop_id   stop_lat   stop_lon               main_cross_street  tract_id
4       47  43.076578 -76.169423  Regional Transportation Center       1.0
   stop_id   stop_lat   stop_lon main_cross_street  tract_id
5       50  43.005051 -76.135737      Loretto Rest     61.01
   stop_id   stop_lat   stop_lon          main_cross_street  tract_id
6       51  42.996453 -76.130486  Nob Hill Apts Building #4     61.02
   stop_id   stop_lat   stop_lon           main_cross_street  tract_id
7       53  43.019482 -76.173644  Glenwood Ave/Strathmore Dr      50.0
    stop_id   stop_lat   stop_lon      main_cross_street  tract_id
20      139  43.031908 -76.148327  S Salina St/Furman St      53.0
    stop_id   stop_lat   stop_lon                    main_cross_street

For 2019, we only use the API on those stops which were not present in 2020. If there is some API misses in getting the tract id for these stops we again use ArcGIS data.

In [384]:
get_all_stop_id_19 = '''
select stop_id, stop_lat, stop_lon, main_cross_street
from Centro_Preprocessing.dbo.FINAL_PREPROCESSED_DATA
where stop_lat is not null and stop_lon is not null and main_cross_street is not null
group by stop_id, stop_lat, stop_lon, main_cross_street;
'''
stop_id_19 = query_run_helper(get_all_stop_id_19)
stop_id_19.head()

,stop_id,stop_lat,stop_lon,main_cross_street
0,1338,43.024825,-76.144920,S Salina St/Elk St
1,18003,42.928122,-76.573401,Murphy Building Genesee St
2,2609,43.062684,-76.022731,Pleasant Dr/Sunset Dr
3,6541,43.052388,-76.132982,Lodi St/Oak St
4,6712,43.052117,-76.129959,Burnet Ave/Elm St


In [132]:
# Get stop ids unique to 2019
stop_ids_20 = df["stop_id"]
unique_stop_ids = pd.DataFrame({})
for row in stop_id_19.itertuples(index=False):
    if(row[0] not in stop_ids_20.values):
        l=[row.stop_id, row.stop_lat, row.stop_lon, row.main_cross_street]
        row = pd.DataFrame([l], columns=stop_id_19.columns.values)
        unique_stop_ids = pd.concat([unique_stop_ids, row], axis=0).reset_index(drop=True)
        

In [133]:
unique_stop_ids.head()

,stop_id,stop_lat,stop_lon,main_cross_street
0,16296,43.077237,-76.000129,Eastview Apartments
1,11605,43.040661,-76.106226,Meadowbrook Dr/Bradford Pky
2,11486,43.078599,-76.140519,Wadsworth St/Hillside St
3,18041,43.072703,-76.214572,NYS Fair - Main Gate
4,7895,43.129955,-76.140279,Centerville Court Apts


In [134]:
unique_tract_ids=get_tract_id(unique_stop_ids)

1
{'address': {'amenity': 'Minoa Village Offices',
             'country': 'United States',
             'country_code': 'us',
             'county': 'Onondaga County',
             'postcode': '13116',
             'road': 'Adams Street',
             'state': 'New York',
             'town': 'Town of Manlius',
             'village': 'Minoa'},
 'boundingbox': ['43.0768471', '43.077574', '-76.0006568', '-76.0002411'],
 'display_name': 'Minoa Village Offices, Adams Street, Minoa, Town of Manlius, '
                 'Onondaga County, New York, 13116, United States',
 'lat': '43.07721975',
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. '
            'https://osm.org/copyright',
 'lon': '-76.00045293810531',
 'osm_id': 277480706,
 'osm_type': 'way',
 'place_id': 157886696}
https://geocoding.geo.census.gov/geocoder/geographies/address?street=Eastview+Apartments&state=New+York&zip=13116&benchmark=Public_AR_Census2020&vintage=Census2020_Census2020&format=json
{'result': {'input': 

In [139]:
# Adding a new column
unique_stop_ids["tract_id"]=unique_tract_ids

In [144]:
# for the stop ids not populated by the API, we use Prof. Baris's data to populate those

stops =unique_stop_ids[unique_stop_ids["tract_id"]==""]["stop_id"]
tracts = []
for stop in stops:
    flg=0
    for row in arcgis.itertuples(index=False):
        if(row[0]==stop):
            tracts.append(row[1]/100)
            flg=1
    if(flg==0):
        tracts.append("")

In [147]:
for (i,stop) in enumerate(stops):
    unique_stop_ids.loc[unique_stop_ids["stop_id"]==stop, "tract_id"]=tracts[i]

This section uses the 2020 data and the unique stop id data we got to populate the 2019 stop id dataset

In [150]:
# populate stop_id_2019 table, merging df(stop_id_2020) and unique_stop_ids dataframe
if os.path.exists('./stop_ids_2020.csv'):
    df=pd.read_csv('./stop_ids_2020.csv')
tract_id=[]
for row in stop_id_19.itertuples(index=False):
    flg=0
    for row2 in df.itertuples(index=False):
        if(row[0]==row2[0]):
            tract_id.append(row2[4])
            flg=1
    if flg==0:
        for row2 in unique_stop_ids.itertuples(index=False):
            if(row[0]==row2[0]):
                tract_id.append(row2[4])
                
stop_id_19["tract_id"]=tract_id
stop_id_19.to_csv('stop_ids_2019.csv', index=False)

Pandas(stop_id=17, stop_lat=43.018504, stop_lon=-76.133335, main_cross_street='Thurber St/Alden St', tract_id=55.0)


### Mini Analysis: Servicable months

We saw that for many routes had a difference in the number of months that they serviced. So we anlayze the number of servicable months across 2019 and 2020 for routes.

In [276]:
#generate servicable months data
q = '''select ROUTE_NAME, count(distinct(DATEPART(month, TRIP_START_TIME))) As servicable_months
from Centro_Preprocessing.dbo.FINAL_PREPROCESSED_DATA
group by ROUTE_NAME;'''
servicable_months_19 = query_run_helper(q)
q_20 = '''select ROUTE_NAME, count(distinct(DATEPART(month, TRIP_START_TIME))) As servicable_months
from Centro_Preprocessing_2020.dbo.FINAL_PREPROCESSED_DATA_2020
group by ROUTE_NAME;'''
servicable_months_20 = query_run_helper(q_20)
merge_df = pd.merge(servicable_months_20, servicable_months_19, how='outer', on=['ROUTE_NAME'])
merge_df
# merge_df.to_csv('servicable_months.csv', index=False)

,ROUTE_NAME,servicable_months_x,servicable_months_y
0,Aub36,4,12.0
1,Aub38,9,12.0
2,AUB50,4,12.0
3,Osw46,12,12.0
4,SU145,3,5.0
5,SU243,7,5.0
6,SU244,7,11.0
7,SU245,6,8.0
8,SU343,4,NaN
9,SU344,11,12.0


### STOP ID Analysis from tracts

As a start, we analyze the top 10 tracts here by number of stops inside the tract. We find these tracts and get data from all stop ids inside those tracts. The data is stored in top_tract_stop_20.csv, top_tract_stop_19.csv

In [234]:
if os.path.exists('./stop_ids_2020.csv'):
    stop_ids_2020=pd.read_csv('./stop_ids_2020.csv')
if os.path.exists('./stop_ids_2019.csv'):
    stop_ids_2019=pd.read_csv('./stop_ids_2019.csv')
    
total_20 = stop_ids_2020.groupby([ 'tract_id'], as_index=False).count().reset_index()[['tract_id', 'stop_id']].sort_values(by='stop_id',ascending=False)
print("TOP TRACTS OF 2020:")
print(total_20[:10])

total_19 = stop_ids_2019.groupby([ 'tract_id'], as_index=False).count().reset_index()[['tract_id', 'stop_id']].sort_values(by='stop_id',ascending=False)
print("TOP TRACTS OF 2019:")
print(total_19[:10])

top_tracts_20 = total_20[:10]["tract_id"]
top_tracts_19 = total_19[:10]["tract_id"]

TOP TRACTS OF 2020:
     tract_id  stop_id
99     144.00      118
38      46.00       93
101    146.00       83
24      32.00       83
16      19.00       76
93     137.01       72
8        9.00       68
25      34.00       66
1        2.00       60
66     111.01       60
TOP TRACTS OF 2019:
    tract_id  stop_id
58       144      129
142       46      116
118       32      108
119       34       91
60       146       90
85        19       82
86         2       80
51    137.01       76
151       55       72
65       151       72


In [254]:
def get_stop_id_data_2020(stop_id):
    try:
        query = '''select MONTH(TRIP_START_TIME) AS month, sum(passengers_on)+sum(passengers_off) AS capacity_20
        from Centro_Preprocessing_2020.dbo.FINAL_PREPROCESSED_DATA_2020
        where stop_id = '{0}' group by MONTH(TRIP_START_TIME) order by MONTH(TRIP_START_TIME);'''.format(stop_id)
        data = pd.read_sql_query(query,cnxn)
        return data
    except Exception as e:
        print(e)
def get_stop_id_data_2019(stop_id):
    try:
        query = '''select MONTH(TRIP_START_TIME) AS month, sum(passengers_on)+sum(passengers_off) AS capacity_19
        from Centro_Preprocessing.dbo.FINAL_PREPROCESSED_DATA
        where stop_id = '{0}' group by MONTH(TRIP_START_TIME) order by MONTH(TRIP_START_TIME);'''.format(stop_id)
        data = pd.read_sql_query(query,cnxn)
        return data
    except Exception as e:
        
        print(e) 
def get_stops_20(tract_id):
    return stop_ids_2020[stop_ids_2020["tract_id"]==tract_id]["stop_id"].unique()
def get_stops_19(tract_id):
    return stop_ids_2019[stop_ids_2019["tract_id"]==tract_id]["stop_id"].unique()

In [255]:
combined_2020_tract = pd.DataFrame({})
for tracts in top_tracts_20:
    for stop in get_stops_20(tracts):
        dt_20=get_stop_id_data_2020(stop)
        dt_19=get_stop_id_data_2019(stop)
        merge_dt = pd.merge(dt_20, dt_19, how='inner', on=['month'])
        merge_dt["tract_id"]=tracts
        merge_dt["stop_id"]=stop
        combined_2020_tract = pd.concat([combined_2020_tract, merge_dt], axis=0).reset_index(drop=True)

# print(combined_2020_tract)
combined_2020_tract.to_csv("top_tract_stop_20.csv", index=False)

In [256]:
combined_2019_tract = pd.DataFrame({})
for tracts in top_tracts_19:
    for stop in get_stops_19(tracts):
        dt_20=get_stop_id_data_2020(stop)
        dt_19=get_stop_id_data_2019(stop)
        merge_dt = pd.merge(dt_20, dt_19, how='inner', on=['month'])
        merge_dt["tract_id"]=tracts
        merge_dt["stop_id"]=stop
        combined_2019_tract = pd.concat([combined_2019_tract, merge_dt], axis=0).reset_index(drop=True)

# print(combined_2019_tract)
combined_2019_tract.to_csv("top_tract_stop_19.csv", index=False)

<ipython-input-256-b6a68f2431b7>:9: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  combined_2019_tract = pd.concat([combined_2019_tract, merge_dt], axis=0).reset_index(drop=True)


# T-test Analysis

As a further analysis we use a two-paired t-test to determine if the non-zero passenger trips of 2019 are statistically any different from the non-zero passenger trips of 2020. This analysis can be found in the file called: month_extensive_t_test.csv

In [462]:
#Steps to calculate t-statistics

# mean1, mean2 = mean(data1), mean(data2)
# 	# calculate standard errors
# 	se1, se2 = sem(data1), sem(data2)
# 	# standard error on the difference between the samples
# 	sed = sqrt(se1**2.0 + se2**2.0)
# 	# calculate the t statistic
# 	t_stat = (mean1 - mean2) / sed
# 	# degrees of freedom
# 	df = len(data1) + len(data2) - 2
# 	# calculate the critical value
# 	cv = t.ppf(1.0 - alpha, df)
# 	# calculate the p-value
# 	p = (1.0 - t.cdf(abs(t_stat), df)) * 2.0

if os.path.exists('./month_extensive_2020.csv'):
    month_extensive_2020=pd.read_csv('./month_extensive_2020.csv')
if os.path.exists('./month_extensive_2019.csv'):
    month_extensive_2019=pd.read_csv('./month_extensive_2019.csv')
month_extensive_2020 = month_extensive_2020[["route_name", "month", "avg_psngr_non_zero", "std_psngr_non_zero", "non_zero_trips","%_zero_trips", "total_trips"]]
month_extensive_2020.columns= ['route_name', 'month', 'avg_psngr_non_zero_20', "std_psngr_non_zero_20", "non_zero_trips_20", "%_zero_trips_20", "total_trips_20"]
month_extensive_2019 = month_extensive_2019[["route_name", "month", "avg_psngr_non_zero", "std_psngr_non_zero", "non_zero_trips", "%_zero_trips", "total_trips"]]
month_extensive_2019.columns= ['route_name', 'month', 'avg_psngr_non_zero_19', "std_psngr_non_zero_19", "non_zero_trips_19", "%_zero_trips_19", "total_trips_19"]
month_extensive = pd.merge(month_extensive_2020,month_extensive_2019, how='inner',on=["route_name", "month"])
month_extensive

,route_name,month,avg_psngr_non_zero_20,std_psngr_non_zero_20,non_zero_trips_20,%_zero_trips_20,total_trips_20,avg_psngr_non_zero_19,std_psngr_non_zero_19,non_zero_trips_19,%_zero_trips_19,total_trips_19
0,AUB50,1,4.83,2.65,24,11.11,27,4.55,3.19,31,3.13,32
1,AUB50,2,5.31,3.29,35,7.89,38,3.52,2.60,29,0.00,29
2,AUB50,3,5.48,4.58,27,18.18,33,4.55,3.94,22,4.35,23
3,AUB50,4,2.43,1.72,7,22.22,9,5.61,3.74,18,0.00,18
4,Aub36,1,19.71,9.32,150,0.00,150,16.41,7.86,107,0.00,107
...,...,...,...,...,...,...,...,...,...,...,...,...
481,SY974,2,37.30,11.70,71,0.00,71,38.12,11.76,69,0.00,69
482,SY974,3,37.31,10.91,52,0.00,52,38.64,10.39,154,0.00,154
483,SY974,10,5.46,4.26,39,2.50,40,38.85,14.55,121,0.00,121
484,SY974,11,3.82,4.21,38,0.00,38,39.87,13.45,82,0.00,82


In [464]:
# function to calculate standard error on the difference between the samples, given by the formula
# se1, se2 = std_dev1/len(n1), std_dev2/len(n2)
# 	sed = sqrt(se1**2.0 + se2**2.0)

def rs(a, b, l1, l2):
    return np.sqrt((a/np.sqrt(l1))**2.0 + (b/np.sqrt(l2))**2.0)
month_extensive['sed'] = month_extensive.apply(lambda row : rs(row['std_psngr_non_zero_20'],
                     row['std_psngr_non_zero_19'], row["non_zero_trips_20"], row["non_zero_trips_19"]), axis = 1)

#function to calculate the t-statistic

def t_stat(mean1, mean2, sed):
    return (mean1 - mean2) / sed
month_extensive['t_stat'] = month_extensive.apply(lambda row : t_stat(row['avg_psngr_non_zero_19'],
                     row['avg_psngr_non_zero_20'], row['sed']), axis = 1)

from scipy.stats import sem
from scipy.stats import t

#function to calculate the p-value
def p_val(l1, l2, alpha, t_stat):
    df=l1+l2-2
    cv = t.ppf(1.0 - alpha, df)
    p = (1.0 - t.cdf(abs(t_stat), df)) * 2.0
    return p
month_extensive['p_value'] = month_extensive.apply(lambda row : p_val(row['non_zero_trips_19'],
                     row['non_zero_trips_20'], 0.05, row['t_stat']), axis = 1)

#function to say if the difference is significant
def yes_no(p, alpha):
    if p > alpha:
        return 'yes'
    else:
        return 'no'
month_extensive['is_similar_sample'] = month_extensive.apply(lambda row : yes_no(row['p_value'], 0.05), axis = 1)
del month_extensive['sed']
month_extensive.to_csv("month_extensive_t_test.csv", index=False)

### STOP ID ANALYSIS

This section furthers the StopID Analysis. We want to analyze the capacity across 2019 & 2020 using monthly averages across tracts.

In [257]:
if os.path.exists('./top_tract_stop_20.csv'):
    top_tract_stop_20=pd.read_csv('./top_tract_stop_20.csv')
if os.path.exists('./top_tract_stop_19.csv'):
    top_tract_stop_19=pd.read_csv('./top_tract_stop_19.csv')

In [259]:
top_tract_20_avg = top_tract_stop_20.groupby(['month', 'tract_id'], as_index=False).mean().reset_index()[['month', 'tract_id', 'capacity_19','capacity_20']].sort_values(by=['tract_id', 'month'])
top_tract_20_avg.columns= ['month', 'tract_id', 'avg_capacity_2019','avg_capacity_2020']

top_tract_20_std = top_tract_stop_20.groupby(['month', 'tract_id'], as_index=False).var().reset_index()[['month', 'tract_id', 'capacity_19','capacity_20']].sort_values(by=['tract_id', 'month'])
top_tract_20_std["capacity_19"]=np.sqrt((top_tract_20_std["capacity_19"]))
top_tract_20_std["capacity_20"]=np.sqrt((top_tract_20_std["capacity_20"]))

top_tract_20_total = top_tract_stop_20.groupby(['month', 'tract_id'], as_index=False).sum().reset_index()[['month', 'tract_id', 'capacity_19','capacity_20']].sort_values(by=['tract_id', 'month'])

top_tract_19_avg = top_tract_stop_19.groupby(['month', 'tract_id'], as_index=False).mean().reset_index()[['month', 'tract_id', 'capacity_19','capacity_20']].sort_values(by=['tract_id', 'month'])
top_tract_19_avg.columns= ['month', 'tract_id', 'avg_capacity_2019','avg_capacity_2020']

top_tract_19_std = top_tract_stop_19.groupby(['month', 'tract_id'], as_index=False).var().reset_index()[['month', 'tract_id', 'capacity_19','capacity_20']].sort_values(by=['tract_id', 'month'])
top_tract_19_std["capacity_19"]=np.sqrt((top_tract_19_std["capacity_19"]))
top_tract_19_std["capacity_20"]=np.sqrt((top_tract_19_std["capacity_20"]))

top_tract_19_total = top_tract_stop_19.groupby(['month', 'tract_id'], as_index=False).sum().reset_index()[['month', 'tract_id', 'capacity_19','capacity_20']].sort_values(by=['tract_id', 'month'])

In [260]:
top_tract_20_avg["std_dev_capacity_2019"] = pd.Series(top_tract_20_std['capacity_19'])
top_tract_20_avg["std_dev_capacity_2020"] = pd.Series(top_tract_20_std['capacity_20'])
top_tract_20_avg["total_capacity_2019"] = pd.Series(top_tract_20_total['capacity_19'])
top_tract_20_avg["total_capacity_2020"] = pd.Series(top_tract_20_total['capacity_20'])

top_tract_19_avg["std_dev_capacity_2019"] = pd.Series(top_tract_19_std['capacity_19'])
top_tract_19_avg["std_dev_capacity_2020"] = pd.Series(top_tract_19_std['capacity_20'])
top_tract_19_avg["total_capacity_2019"] = pd.Series(top_tract_19_total['capacity_19'])
top_tract_19_avg["total_capacity_2020"] = pd.Series(top_tract_19_total['capacity_20'])

In [261]:
top_tract_20_avg.to_csv("top_tract_2020.csv", index=False)
top_tract_19_avg.to_csv("top_tract_2019.csv", index=False)

### STOP ID ANALYSIS VISUALIZATION

### 2020 Analysis

In [275]:
# The following code block creates the necessary visualizations for stop id analysis

#####################################################################
%matplotlib inline
import ipywidgets as widgets
from matplotlib.pyplot import *

grp_plot_output = widgets.Output()
if os.path.exists('./top_tract_2020.csv'):
    top_tract_2020=pd.read_csv('./top_tract_2020.csv')
if os.path.exists('./top_tract_2019.csv'):
    top_tract_2019=pd.read_csv('./top_tract_2019.csv')
# The dropdown values are defined below
yr=["Choose year", "2019", "2020"]
all_groups = top_tract_2019["tract_id"].unique()
all_types = ["Compare Average Capacity", "Compare Total Capacity"]

def get_tracts():
    if dropdown_yr.value=="2020":
        return top_tract_2020["tract_id"].unique()
    elif dropdown_yr.value=="2019":
        return top_tract_2019["tract_id"].unique()


# The code below we create the dropdown widget with above values
dropdown_yr = widgets.Dropdown(options=yr, description="Year:")
dropdown_tract = widgets.Dropdown(options=["Choose Tract"], description="Tract:")
dropdown_types = widgets.Dropdown(options=all_types, description="Comparison Type:")


# The common function to handle changes in dropdowns and refreshing the plot
def filtering(tract_id, type_name):
    grp_plot_output.clear_output()
    if dropdown_yr.value=="2020":
        df=top_tract_2020[top_tract_2020["tract_id"]==tract_id]
    else:
        df=top_tract_2019[top_tract_2019["tract_id"]==tract_id]
    
    with grp_plot_output:
        if type_name == "Compare Average Capacity":
            df.plot.bar(x = 'month', y = ['avg_capacity_2020', 'avg_capacity_2019'], rot = 40, 
                        title="Year: {0} \n Tract_ID: {1}".format(dropdown_yr.value, dropdown_tract.value))
            show()
        else:
            df.plot.bar(x = 'month', y = ['total_capacity_2020', 'total_capacity_2019'], rot = 40,
                       title="Year: {0} \n Tract_ID: {1}".format(dropdown_yr.value, dropdown_tract.value))
            show()
def dropdown_yr_eventhandler(change):
    dropdown_tract.options=get_tracts()
            
def dropdown_tract_eventhandler(change):
    filtering(change.new, dropdown_types.value)

def dropdown_types_eventhandler(change):
    filtering(dropdown_tract.value, change.new)

dropdown_yr.observe(dropdown_yr_eventhandler, names='value')
dropdown_tract.observe(dropdown_tract_eventhandler, names='value')
dropdown_types.observe(dropdown_types_eventhandler, names ='value')

# displaying the widgets
display(dropdown_yr)
display(dropdown_tract)
display(dropdown_types)
display(grp_plot_output)

Dropdown(description='Year:', options=('Choose year', '2019', '2020'), value='Choose year')

Dropdown(description='Tract:', options=('Choose Tract',), value='Choose Tract')

Dropdown(description='Comparison Type:', options=('Compare Average Capacity', 'Compare Total Capacity'), value…

Output()

### All tracts & stops visualization

In [273]:
# The following code block creates the necessary visualizations for stop id analysis

#####################################################################
%matplotlib inline
import ipywidgets as widgets
from matplotlib.pyplot import *

grp_plot_output = widgets.Output()
if os.path.exists('./top_tract_stop_20.csv'):
    top_tract_stop_20=pd.read_csv('./top_tract_stop_20.csv')
if os.path.exists('./top_tract_stop_19.csv'):
    top_tract_stop_19=pd.read_csv('./top_tract_stop_19.csv')
# The dropdown values are defined below
yr=["2019", "2020"]
all_groups = top_tract_2019["tract_id"].unique()
all_stops=top_tract_stop_19["stop_id"].unique()

def get_tracts():
    if dropdown_yr.value=="2020":
        return top_tract_stop_20["tract_id"].unique()
    else:
        return top_tract_stop_19["tract_id"].unique()

def get_stops():
    if dropdown_yr.value=="2020":
        return top_tract_stop_20[top_tract_stop_20["tract_id"]==dropdown_tract.value]["stop_id"].unique()
    else:
        return top_tract_stop_19[top_tract_stop_19["tract_id"]==dropdown_tract.value]["stop_id"].unique()

# The code below we create the dropdown widget with above values
dropdown_yr = widgets.Dropdown(options=yr, description="Year:")
dropdown_tract = widgets.Dropdown(options=["Choose tract"], description="Tract:")
dropdown_types = widgets.Dropdown(options=["Choose Stops"], description="Stop ID:")


# The common function to handle changes in dropdowns and refreshing the plot
def filtering(tract_id, type_name):
    grp_plot_output.clear_output()
    if dropdown_yr.value=="2020":
        df=top_tract_stop_20[(top_tract_stop_20["tract_id"]==dropdown_tract.value) & (top_tract_stop_20["stop_id"]==dropdown_types.value)]
    else:
        df=top_tract_stop_19[(top_tract_stop_19["tract_id"]==dropdown_tract.value) & (top_tract_stop_19["stop_id"]==dropdown_types.value)]
    with grp_plot_output:
        df.plot.bar(x = 'month', y = ['capacity_20', 'capacity_19'], rot = 40, 
                    title="Year: {0} \n Tract_ID: {1} \n Stop_ID:{2}".format(dropdown_yr.value, dropdown_tract.value, dropdown_types.value))
        show()
def dropdown_yr_eventhandler(change):
    dropdown_tract.options=get_tracts()
            
def dropdown_tract_eventhandler(change):
    dropdown_types.options=get_stops()

def dropdown_types_eventhandler(change):
    filtering(dropdown_tract.value, change.new)

dropdown_yr.observe(dropdown_yr_eventhandler, names='value')
dropdown_tract.observe(dropdown_tract_eventhandler, names='value')
dropdown_types.observe(dropdown_types_eventhandler, names ='value')

# displaying the widgets
display(dropdown_yr)
display(dropdown_tract)
display(dropdown_types)
display(grp_plot_output)

Dropdown(description='Year:', options=('2019', '2020'), value='2019')

Dropdown(description='Tract:', options=('Choose tract',), value='Choose tract')

Dropdown(description='Stop ID:', options=('Choose Stops',), value='Choose Stops')

Output()